In [ ]:
!pip install ultralytics
!pip install gdown

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 28.4 MB/s eta 0:00:00


In [ ]:
import os
import xml.etree.ElementTree as ET
from PIL import Image
from ultralytics import YOLO

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


In [ ]:
!mkdir -p datasets/IR-LPR

!gdown --id 1XtZ-XQ8ImNFf40D-bFqTm0UVFqNKhbLi -O datasets/IR-LPR/train.zip
!gdown --id 1hwz6X-Zp7JpJL35K6P3z7k6O_PTXhUcT -O datasets/IR-LPR/val.zip
!gdown --id 1pe4_HgXb9dctFGJXVNlyNcKSXZeht0lX -O datasets/IR-LPR/test.zip

!unzip -q datasets/IR-LPR/train.zip -d datasets/IR-LPR/
!unzip -q datasets/IR-LPR/val.zip -d datasets/IR-LPR/
!unzip -q datasets/IR-LPR/test.zip -d datasets/IR-LPR/

/usr/local/lib/python3.12/dist-packages/gdown/__main__.py:140: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From (original): https://drive.google.com/uc?id=1XtZ-XQ8ImNFf40D-bFqTm0UVFqNKhbLi
From (redirected): https://drive.google.com/uc?id=1XtZ-XQ8ImNFf40D-bFqTm0UVFqNKhbLi&confirm=t&uuid=e358d5fa-69d7-466f-9ca2-3a21fafc83a1
To: /content/datasets/IR-LPR/train.zip
100% 8.89G/8.89G [01:40<00:00, 88.7MB/s]
/usr/local/lib/python3.12/dist-packages/gdown/__main__.py:140: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From (original): https://drive.google.com/uc?id=1hwz6X-Zp7JpJL35K6P3z7k6O_PTXhUcT
From (redirected): https://drive.google.com/uc?id=1hwz6X-Zp7JpJL35K6P3z7k6O_PTXhUcT&confirm=t&uuid=34e58c42-326f-4151-9eb0-eb67c4156d9e
To: /content

In [ ]:
xml_path = "datasets/IR-LPR/train/day_11469.xml"

tree = ET.parse(xml_path)
root = tree.getroot()

print("=== Root tag:", root.tag)

# ابعاد تصویر
size = root.find("size")
if size is not None:
    w = int(size.find("width").text)
    h = int(size.find("height").text)
    print("Image size:", w, "x", h)

# همه‌ی object ها
for obj in root.findall("object"):
    name = obj.find("name").text
    bbox = obj.find("bndbox")
    xmin = bbox.find("xmin").text
    ymin = bbox.find("ymin").text
    xmax = bbox.find("xmax").text
    ymax = bbox.find("ymax").text

    print(f"Class: {name}  BBox: ({xmin}, {ymin}, {xmax}, {ymax})")


=== Root tag: annotation
Class: کل ناحیه پلاک  BBox: (405, 287, 529, 322)
Class: 7  BBox: (419, 290, 433, 318)
Class: 3  BBox: (433, 290, 446, 318)
Class: ه‍  BBox: (448, 290, 468, 317)
Class: 1  BBox: (468, 289, 480, 317)
Class: 3  BBox: (479, 289, 493, 317)
Class: 8  BBox: (492, 289, 505, 316)
Class: 1  BBox: (507, 293, 519, 317)
Class: 0  BBox: (518, 298, 528, 314)


In [ ]:
def voc_to_yolo(xml_file, txt_file):
    """تبدیل یک فایل XML به فرمت YOLO (فقط پلاک‌ها)"""
    tree = ET.parse(xml_file)
    root = tree.getroot()

    # مسیر فایل تصویر
    img_file = xml_file.replace(".xml", ".jpg")
    if not os.path.exists(img_file):
        print(f"Image not found, skipping: {img_file}")
        return

    # گرفتن ابعاد تصویر از خود JPG
    with Image.open(img_file) as im:
        w, h = im.size

    lines = []
    for obj in root.findall("object"):
        cls = obj.find("name").text.strip()
        if cls != "کل ناحیه پلاک":
            continue  # فقط پلاک‌ها را نگه می‌داریم

        bbox = obj.find("bndbox")
        xmin = float(bbox.find("xmin").text)
        ymin = float(bbox.find("ymin").text)
        xmax = float(bbox.find("xmax").text)
        ymax = float(bbox.find("ymax").text)

        # تبدیل به فرمت YOLO (نرمالایز)
        x_center = (xmin + xmax) / 2.0 / w
        y_center = (ymin + ymax) / 2.0 / h
        bw = (xmax - xmin) / w
        bh = (ymax - ymin) / h

        lines.append(f"0 {x_center} {y_center} {bw} {bh}")

    # نوشتن فایل txt
    with open(txt_file, "w") as f:
        f.write("\n".join(lines))


def convert_dataset(src_dir, out_images, out_labels):
    """تبدیل کل پوشه (train/validation/test)"""
    os.makedirs(out_images, exist_ok=True)
    os.makedirs(out_labels, exist_ok=True)

    for file in os.listdir(src_dir):
        if file.endswith(".xml"):
            xml_path = os.path.join(src_dir, file)
            jpg_path = xml_path.replace(".xml", ".jpg")
            if not os.path.exists(jpg_path):
                print(f"Image not found, skipping: {jpg_path}")
                continue

            # کپی عکس به پوشه images
            os.system(f"cp '{jpg_path}' '{out_images}/'")

            # ساخت فایل txt در پوشه labels
            txt_name = file.replace(".xml", ".txt")
            txt_path = os.path.join(out_labels, txt_name)
            voc_to_yolo(xml_path, txt_path)


# مسیر دیتاست اصلی
base = "datasets/IR-LPR"

# تبدیل train
convert_dataset(f"{base}/train", f"{base}/images/train", f"{base}/labels/train")
# تبدیل validation
convert_dataset(f"{base}/validation", f"{base}/images/val", f"{base}/labels/val")
# تبدیل test
convert_dataset(f"{base}/test", f"{base}/images/test", f"{base}/labels/test")

print("Dataset converted to YOLO format successfully!")


Dataset converted to YOLO format successfully!


In [ ]:
base_dir = "/content/datasets/IR-LPR"

yaml_content = f"""
train: {base_dir}/images/train
val: {base_dir}/images/val
test: {base_dir}/images/test

nc: 1
names: ['plate']
"""

yaml_path = os.path.join(base_dir, "data.yaml")
with open(yaml_path, "w", encoding="utf-8") as f:
    f.write(yaml_content)

print(f"data.yaml ساخته شد در مسیر: {yaml_path}")

model = YOLO("yolov9m")

model.train(
    data=yaml_path,
    epochs=20,
    imgsz=640,
    batch=16,
    lr0=0.01,
    project="/content/yolo_plate",
    name="train1",
    exist_ok=True
)

# بعد از آموزش می‌توانیم نتایج را روی تصاویر تست ببینیم:
results = model.predict(source=f"{base_dir}/images/test", save=True)

data.yaml ساخته شد در مسیر: /content/datasets/IR-LPR/data.yaml


Ultralytics 8.3.185 🚀 Python-3.12.11 torch-2.8.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/datasets/IR-LPR/data.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=20, erasing=0.4, exist_ok=True, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov9m.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=train1, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=100, perspective=0.0, plots=True, pose=12.0, pret

Overriding model.yaml nc=80 with nc=1

                   from  n    params  module                                       arguments                     
  0                  -1  1       928  ultralytics.nn.modules.conv.Conv             [3, 32, 3, 2]                 
  1                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  2                  -1  1    171648  ultralytics.nn.modules.block.RepNCSPELAN4    [64, 128, 128, 64, 1]         
  3                  -1  1    276960  ultralytics.nn.modules.block.AConv           [128, 240]                    
  4                  -1  1    629520  ultralytics.nn.modules.block.RepNCSPELAN4    [240, 240, 240, 120, 1]       
  5                  -1  1    778320  ultralytics.nn.modules.block.AConv           [240, 360]                    
  6                  -1  1   1414080  ultralytics.nn.modules.block.RepNCSPELAN4    [360, 360, 360, 180, 1]       
  7                  -1  1   1556160  ultralytics

 18                  -1  1   1480320  ultralytics.nn.modules.block.RepNCSPELAN4    [544, 360, 360, 180, 1]       
 19                  -1  1    778080  ultralytics.nn.modules.block.AConv           [360, 240]                    
 20             [-1, 9]  1         0  ultralytics.nn.modules.conv.Concat           [1]                           
 21                  -1  1   2627040  ultralytics.nn.modules.block.RepNCSPELAN4    [720, 480, 480, 240, 1]       
 22        [15, 18, 21]  1   4637539  ultralytics.nn.modules.head.Detect           [1, [240, 360, 480]]          
YOLOv9m summary: 348 layers, 20,159,043 parameters, 20,159,027 gradients, 77.5 GFLOPs

Transferred 901/907 items from pretrained weights
Freezing layer 'model.22.dfl.conv.weight'
AMP: running Automatic Mixed Precision (AMP) checks...


AMP: checks passed ✅
train: Fast image access ✅ (ping: 0.0±0.0 ms, read: 90.6±21.8 MB/s, size: 659.7 KB)


...

      20/20      8.19G     0.7136     0.3497     0.9201         16        640: 100%|██████████| 917/917 [09:43<00:00,  1.57it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 67/67 [00:37<00:00,  1.80it/s]

                   all       2120       2494      0.936      0.938      0.973       0.78



20 epochs completed in 3.488 hours.
Optimizer stripped from /content/yolo_plate/train1/weights/last.pt, 40.8MB
Optimizer stripped from /content/yolo_plate/train1/weights/best.pt, 40.8MB

Validating /content/yolo_plate/train1/weights/best.pt...
Ultralytics 8.3.185 🚀 Python-3.12.11 torch-2.8.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
YOLOv9m summary (fused): 151 layers, 20,013,715 parameters, 0 gradients, 76.5 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 67/67 [00:34<00:00,  1.91it/s]


                   all       2120       2494      0.936      0.937      0.973      0.781
Speed: 0.2ms preprocess, 8.2ms inference, 0.0ms loss, 2.7ms postprocess per image
Results saved to /content/yolo_plate/train1

WARNING ⚠️ 
inference results will accumulate in RAM unless `stream=True` is passed, causing potential out-of-memory
errors for large sources or long-running streams and videos. See https://docs.ultralytics.com/modes/predict/ for help.

Example:
    results = model(source=..., stream=True)  # generator of Results objects
    for r in results:
        boxes = r.boxes  # Boxes object for bbox outputs
        masks = r.masks  # Masks object for segment masks outputs
        probs = r.probs  # Class probabilities for classification outputs

image 1/4176 /content/datasets/IR-LPR/images/test/day_00010.jpg: 480x640 2 plates, 65.6ms
image 2/4176 /content/datasets/IR-LPR/images/test/day_00019.jpg: 640x480 1 plate, 66.0ms
image 3/4176 /content/datasets/IR-LPR/images/test/day_00026.jp

In [ ]:
base_dir = "/content/datasets/IR-LPR"

yaml_content = f"""
train: {base_dir}/images/train
val: {base_dir}/images/val
test: {base_dir}/images/test

nc: 1
names: ['plate']
"""

yaml_path = os.path.join(base_dir, "data.yaml")
with open(yaml_path, "w", encoding="utf-8") as f:
    f.write(yaml_content)

print(f"data.yaml ساخته شد در مسیر: {yaml_path}")

model = YOLO("/content/best.pt")  # یا best.pt

model.train(
    data=yaml_path,
    epochs=30,          # تعداد epochs جدید
    imgsz=640,
    batch=16,
    lr0=0.001,          # معمولاً LR را کمتر می‌کنیم
    project="/content/yolo_plate",
    name="train2",
    exist_ok=True
)

data.yaml ساخته شد در مسیر: /content/datasets/IR-LPR/data.yaml
Ultralytics 8.3.189 🚀 Python-3.12.11 torch-2.8.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/datasets/IR-LPR/data.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=30, erasing=0.4, exist_ok=True, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.001, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=/content/best.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=train2, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap

ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7aaf2839cc80>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
          0.048048, 